# Analysis of Integrated multi-omics single cell atlas of the human retina

In [ ]:
import sys
import scanpy as sc
import anndata as ad
import numpy as np

In [ ]:
sys.version

'3.10.5 (main, Jul 19 2022, 08:28:45) [GCC 6.2.0]'

In [ ]:
adata = sc.read_h5ad("d0183df5-815d-48c2-bcfe-fbf9b716505c.h5ad")